setup: open trades database and table epex_12_20_12_13

In [4]:
import sqlite3

In [5]:
import pandas as pd

In [15]:
con = sqlite3.connect("trades.sqlite")
trades = pd.read_sql_query("SELECT * from epex_12_20_12_13", con)
print(trades.head())
con.close()
print(trades.size, trades.shape)

        id  quantity  price  side    strategy
0  trade_1         5   20.0   buy  strategy_1
1  trade_2        17   10.0  sell  strategy_1
2  trade_3        15    8.0   buy  strategy_1
3  trade_4        13   35.0  sell  strategy_2
4  trade_5        23   11.0  sell  strategy_2
30 (6, 5)
